In [ ]:
import os
import pickle
import tsplib95
import networkx as nx
from networkx.algorithms.approximation.traveling_salesman import *

In [ ]:
# load optimal tour lengths
opt_tour_lengths = dict()
with open("tsp_dataset/tsp.soln") as file:
    for line in file:
        name, _, opt_tour, *_ = line.split()
        opt_tour_lengths[name] = int(opt_tour)

def get_opt_tour_length(name):
    return opt_tour_lengths[name]

In [ ]:
def get_tsp_graph(name) -> nx.Graph:
    # load from pkl file if exists
    graph_file_path = os.path.join("tsp_dataset", f"{name}.graph.pkl")
    if os.path.exists(graph_file_path):
        print(f"Loading {name} from {graph_file_path}")
        with open(graph_file_path, "rb") as file:
            G = pickle.load(file)
        return G

    tsp_file_path = os.path.join("tsp_dataset", f"{name}.tsp")
    print(f"Loading {name} from {tsp_file_path}")
    problem = tsplib95.load(tsp_file_path)
    G = problem.get_graph()

    if problem.edge_weight_type == "EXPLICIT" and problem.edge_weight_format == "FULL_MATRIX":
        for u in G.nodes:
            for v in G.nodes:
                if u > v:
                    G.remove_edge(u, v)

    loop_edges = list(nx.selfloop_edges(G))
    G.remove_edges_from(loop_edges)

    n, m = G.number_of_nodes(), G.number_of_edges()
    assert n * (n - 1) // 2 == m

    # save to pkl file
    with open(graph_file_path, "wb") as file:
        pickle.dump(G, file)

    return G

In [ ]:
tsp_graphs = dict()
for tsp_file in os.listdir("tsp_dataset"):
    name, file_extn = os.path.splitext(tsp_file)
    if file_extn == ".tsp":
        print(name)
        problem = tsplib95.load(os.path.join("tsp_dataset", f"{name}.tsp"))
        
        G = get_tsp_graph(name)
        G = G.to_undirected()
        print(G.graph, G.is_directed())

        tour = christofides(G)
        print("tour:", tour)
        tour_length = problem.trace_tours([tour])[0]
        apprx_ratio = tour_length / get_opt_tour_length(name)
        print(tour_length, apprx_ratio)
        print()

In [ ]:
problem = tsplib95.load("tsp_dataset/a280.tsp")
opt_tour_length = 2020

In [ ]:
problem.as_dict()

In [ ]:
G = problem.get_graph()

In [ ]:
G.graph

In [ ]:
G.number_of_nodes(), G.number_of_edges()

In [ ]:
list(G.edges(data=True))

In [ ]:
G.number_of_nodes(), G.number_of_edges()

In [ ]:
n, m = G.number_of_nodes(), G.number_of_edges()
print(n, m)

In [ ]:
assert n * (n - 1) // 2 == m

In [ ]:
tour = christofides(G)
print("tour:", tour)
tour_length = problem.trace_tours([tour])[0]
print("tour length:", tour_length)
print("approximation ratio:", tour_length / opt_tour_length)